<a href="https://colab.research.google.com/github/partizanos/computational_finance/blob/master/TP12_Comp_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Series 12
Introduction to Computational Finance
DIMITRIS PROIOS

## Bonds
Using the same bond as last week, and assuming the required yield is 10%: A bond with
3 years maturity, a face value of 100$ and coupons of 10% paid semi-annually:

|time [years] | coupon [CHF] 
|---|---| 
| 0.5 | 5
| 1 | 5
| 1.5 | 5
| 2 | 5
|2.5 | 5
| 3 | 5


- What is the present value of the bond ?

**Answer**

Value of bonds can be calculated with this formula:
$B = (\sum c e^{rt}) + P e^{rt}$

By replacing with the data of our table the formula becomes:

$B = 5 e^{-0.1 * 0.5} + 5 e^{-0.1* 1} +5 e^{-0.1 * 1.5} +5 e^{-0.1* 2} +5 e^{-0.1 * 2.5}  + 105 e^{-0.1 * 3}$


In [34]:
import numpy as np

discount = lambda n, rate: np.power(np.e, (-n * rate))

coupon_values = lambda coupon, periods: sum(
    [coupon * discount(n, rate) for n in periods]
)

def total_present_value(face_value, coupon, periods, rate):
    cv = coupon_values(coupon, periods)
    print("coupon values: ", cv )
    last_payment = face_value * discount(periods[-1], rate)
    print("last_payment: ", last_payment)
    total_pv = cv + last_payment 
    print("total_pv: ", total_pv)
    
    return total_pv 


face_value = 100
coupon = 5
periods = np.linspace(0.5, 3, 6).tolist()
rate = 0.05

total_present_value(face_value, coupon, periods, rate)


coupon values:  27.51162559951606
last_payment:  86.07079764250578
total_pv:  113.58242324202183


113.58242324202183

- Compute the duration and the convexity of the bond.


In [1]:
def duration(bond_facevalue, time):
  retun res
#Duration of bond
dur_bond=100


print("Duration of bond ", dur_bond)

Duration of bond  100


- Estimate (with and without convexity) what would be the effect of a x% parallel
shift of the yield curve when x equals 0.1% and 1%.
- Compare your answer to the exact values.


If computing the exact value is so simple, why bothering using an approximation through
duration ?